In [ ]:
import requests
import os
from bs4 import BeautifulSoup
def getHtml(url):
    try:
        #요청 헤더 설정 : 브라우저 정보
        req_header = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
        }

        # requests 의 get() 함수 호출하기 
        res=requests.get(url,headers=req_header)
        res.encoding = 'utf-8'
        res.raise_for_status()

        # 응답(response)이 OK 이면 text 추출
        html = res.text
        return html
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return "Error"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "Error"   
url = getHtml("https://huggingface.co/datasets/ealvaradob/phishing-dataset")
print(url)
        

In [ ]:
%pip install webdriver-manager
%pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
def get_rendered_html_with_selenium(url,headless=True, wait_time=10):
    
    
    options = webdriver.ChromeOptions()

    if headless:
        options.add_argument('headless') # UI 없이 백그라운드에서 실행
        options.add_argument('--no-sandbox') # 샌드박스 비활성화 (Docker 등 환경에서 필요)
        options.add_argument('--disable-dev-shm-usage') # /dev/shm 문제 해결 (Docker 등 환경에서 필요)
        options.add_argument('--disable-gpu') # GPU 사용 안 함 (일부 환경에서 필요)

    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
    


    driver = None
    try:
        print(f"Opening URL: {url}")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(url)
    
        WebDriverWait(driver, wait_time).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        print("Page loaded successfully.")

        # JavaScript 실행 후 렌더링된 HTML 소스 코드 가져오기
        rendered_html = driver.page_source
        return rendered_html

    except TimeoutException:
        print(f"Error: Page loading timed out after {wait_time} seconds for URL: {url}")
        return None
    except WebDriverException as e:
        print(f"WebDriver error for URL {url}: {e}")
        print("Please check if ChromeDriver is correctly installed and its path is set.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred for URL {url}: {e}")
        return None
    finally:
        if driver:
            driver.quit() # 브라우저 닫기

# --- 사용 예시 ---
if __name__ == "__main__":
    # 1. 웹 드라이버 경로 지정 (선택 사항 - PATH에 없거나 특정 버전을 사용하고 싶을 때)
    #    예: Windows에서는 r'C:\Users\YourUser\Downloads\chromedriver.exe'
    #    예: macOS/Linux에서는 '/usr/local/bin/chromedriver'
    #    또는 실행 스크립트와 같은 디렉토리에 드라이버를 두었다면 None으로 두어도 됩니다.
    my_driver_path = None # 'chromedriver'가 PATH에 있다면 None으로 두세요.
                           # 직접 경로를 지정하려면 'C:/path/to/chromedriver.exe' 처럼 변경

    # 테스트할 URL
    test_urls = [
        "https://www.google.com",  # 일반적인 웹사이트
        "https://react.dev/",      # SPA 예시 (JS로 동적 로드)
        "https://www.naver.com/",  # JS가 많은 웹사이트
        "http://invalid.url.com",   # 존재하지 않는 URL 테스트
    ]

    for url_to_check in test_urls:
        print(f"\n--- Processing URL: {url_to_check} ---")
        html_content = get_rendered_html_with_selenium(url_to_check,headless=True)

        if html_content:
            # HTML 콘텐츠의 일부만 출력하여 확인 (너무 길 수 있으므로)
            print(f"Successfully retrieved HTML (first 500 chars):")
            print(html_content[:500])
            print("...")
        else:
            print(f"Failed to retrieve HTML for {url_to_check}")

    print("\nScript finished.")